# BVC Stocks database
La idea de este proyecto es crear una base de datos que almacene el precio de las acciones que cotizan en la bolsa de valores de Colombia (BVC) tanto del mercado local como  del mercado global colombiano.

In [ ]:
# Carga de librerias
import pandas as pd
import sqlite3
import os
import time
import schedule
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
# Preferencias de navegador para mercado local
download_dir_ml = os.path.join(os.getcwd(),"datos_ml")
opciones_ml = webdriver.EdgeOptions()
prefs_ml = {"download.default_directory":download_dir_ml}
opciones_ml.add_experimental_option("prefs",prefs_ml)

In [ ]:
# Direcciones

ml ="https://www.bvc.com.co/mercado-local-en-linea?tab=renta-variable_mercado-local"
mgc = "https://www.bvc.com.co/mercado-local-en-linea?tab=renta-variable_mercado-global-colombiano"

# Fecha actual y formato del archivo
ahora = datetime.now()
    
year = ahora.year

if ahora.month<10:
    month=f"0{ahora.month}"
else:
    month = ahora.month

if ahora.day<10:
    day=f"0{ahora.day}"
else:
    day = ahora.day

nombre_ml = f"RVLocal_{year}{month}{day}.csv"
nombre_mgc = f"RVMGC_{year}{month}{day}.csv"

In [ ]:
# Descarga del archivo del mercado local (ml)

def download_ml():
    try:
        driver_ml = webdriver.Edge(options=opciones_ml)
        driver_ml.get(ml)
        
        time.sleep(10)

        download_button_ml = driver_ml.find_element(by=By.XPATH, value='//*[@id="__next"]/div/div[3]/div[2]/div[1]/div[2]/div[2]/div/div/div[2]/div[2]/div/div/section/div[5]/div[2]/div/button')
        download_button_ml.click()

        time.sleep(10)

        if os.path.exists(os.path.join(download_dir_ml,nombre_ml)):
            print("El archivo de Mercado Local se ha descargado correctamente.")
        else:
            print("El archivo de Mercado Local no se ha descargado correctamente.")


    except:
        print("No se ha podido descargar el archivo, revise si hoy es día festivo.")

    else:
        weekdays = ["Lunes","Martes","Miércoles","Jueves","Viernes","Sábado","Domingo"]
        dia = weekdays[ahora.weekday()]
        if dia =="Sábado" or dia =="Domingo":
            print(f"Hoy es {dia}, no es dia hábil, por tanto no puede descargar el archivo.")
        

    finally:
        driver_ml.quit()

download_ml()

In [ ]:
# El siguiente bloque permite programar la descarga del archivo del mercado local a las 5:00 p.m (1 hora después del cierre del mercado)
# para correr este bloque se debe comentar o eliminar la última linea del bloque anterior
# y descomentar el que se encuentra aqui abajo.

""" 
schedule.every().day.at("17:00").do(download_ml)

while True:
    schedule.run_pending()
    time.sleep(1) 
"""

In [ ]:
# Preferencias del navegador para mercado global colombiano
download_dir_mgc = os.path.join(os.getcwd(),"datos_mgc")
opciones_mgc = webdriver.EdgeOptions()
prefs_mgc = {"download.default_directory":download_dir_mgc}
opciones_mgc.add_experimental_option("prefs",prefs_mgc)

In [ ]:
# Descarga del archivo del mercado global colombiano (mgc)

def download_mgc():

    try:
        driver_mgc = webdriver.Edge(options=opciones_mgc)
        driver_mgc.get(mgc)
        
        time.sleep(10)

        download_button_mgc = driver_mgc.find_element(by=By.XPATH, value='//*[@id="__next"]/div/div[3]/div[2]/div[1]/div[2]/div[2]/div/div/div[2]/div[2]/div/div/section/div[5]/div[2]/div/button')
        download_button_mgc.click()

        time.sleep(10)

        if os.path.exists(os.path.join(download_dir_mgc,nombre_mgc)):
            print("El archivo de Mercado Local se ha descargado correctamente.")
        else:
            print("El archivo de Mercado Local no se ha descargado correctamente.")


    except:
        print("No se ha podido descargar el archivo, revise si hoy es día festivo.")

    else:
        weekdays = ["Lunes","Martes","Miércoles","Jueves","Viernes","Sábado","Domingo"]
        dia = weekdays[ahora.weekday()]
        if dia =="Sábado" or dia =="Domingo":
            print(f"Hoy es {dia}, no es dia hábil, por tanto no puede descargar el archivo.")
        

    finally:
        driver_mgc.quit()
        
download_mgc()

In [ ]:
# Igualmente pueden programar la descarga del archivo del mercado global a las 5:00 p.m (1 hora después del cierre del mercado)
# con este bloque de código pero se debe comentar o eliminar la última linea del bloque anterior 
# y quitar el comentario del bloque de código que ven aquí abajo.

""" 
schedule.every().day.at("17:00").do(download_mgc)

while True:
    schedule.run_pending()
    time.sleep(1) 
"""

In [ ]:
## Procesar los datos del archivo csv del mercado local en un dataframe
df_ml = pd.read_csv(
    filepath_or_buffer=os.path.join(download_dir_ml,nombre_ml),
    sep=";",encoding="utf-8",na_values="-", header=None,skiprows=1, decimal=",",
    names=[
        "Nemotécnico","Último_precio","Variación_porcentual","Volúmenes","Cantidad",
        "Variación_absoluta","Precio_apertura","Precio_máximo","Precio_mínimo","Precio_promedio","Emisor"],
        usecols=[i for i in range(11)])
df_ml["Fecha"] = f"{ahora.day}/{month}/{year}"
df_ml.head()

In [ ]:
# Procesando los datos del archivo csv del mercado global colombiano en un dataframe
df_mgc = pd.read_csv(filepath_or_buffer=os.path.join(download_dir_mgc,nombre_mgc),
                     sep=";", encoding="utf-8", na_values="-", header=None, skiprows=1, decimal=",",
                     names=["Nemotécnico","Último_precio","Variación_porcentual","Volúmenes","Cantidad","Variación_absoluta","Emisor"],
                     usecols=[i for i in range(7)])
df_mgc["Fecha"]=f"{ahora.day}/{month}/{year}"
df_mgc.head()

In [ ]:
## Creando la base de datos

connection = sqlite3.connect("stock_prices.db")

cursor = connection.cursor()

# Creado la tabla mercado_local
cursor.execute("""
    CREATE TABLE IF NOT EXISTS mercado_local (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    Nemotécnico TEXT NOT NULL,
    Último_precio REAL,
    Variación_porcentual REAL,
    Volúmenes REAL,
    Cantidad REAL,
    Variación_absoluta REAL,
    Precio_apertura REAL,
    Precio_máximo REAL,
    Precio_mínimo REAL,
    Precio_promedio REAL,
    Emisor TEXT,
    Fecha TEXT);
    
""")

connection.commit()

connection.close()

In [ ]:
# Creando la tabla mercado_global_colombiano
connection = sqlite3.connect("stock_prices.db")
cursor = connection.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS mercado_global_colombiano(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    Nemotécnico TEXT NOT NULL,
    Último_precio REAL,
    Variación_porcentual REAL,
    Volúmenes REAL,
    Cantidad REAL,
    Variación_absoluta REAL,
    Emisor TEXT,
    Fecha TEXT);"""
)

connection.commit()
connection.close()

In [ ]:
# Cargando los datos en la base de datos (mercado local)
conn = sqlite3.connect("stock_prices.db")
df_ml.to_sql(name="mercado_local",con=conn,if_exists="append",index=False)

conn.close()

In [ ]:
# Cargando los datos en la base de datos (mercado global colombiano)
conn = sqlite3.connect("stock_prices.db")
df_mgc.to_sql(name="mercado_global_colombiano",con=conn,if_exists="append",index=False)

conn.close()

In [ ]:
# Verificando los datos cargados en la base de datos (mercado local)
connection = sqlite3.connect("stock_prices.db")

cursor = connection.cursor()

cursor.execute("SELECT * FROM mercado_local LIMIT 5")

resultados = cursor.fetchall()

columnas = [name[0] for name in cursor.description]

print(columnas)
for fila in resultados:
    print(fila)
    
cursor.close()
connection.close()

In [ ]:
# Verificando los datos cargados en la base de datos (mercado global colombiano)
connection = sqlite3.connect("stock_prices.db")

cursor = connection.cursor()

cursor.execute("SELECT * FROM mercado_global_colombiano LIMIT 5")

resultados = cursor.fetchall()

columnas = [name[0] for name in cursor.description]

print(columnas)
for fila in resultados:
    print(fila)
    
cursor.close()
connection.close()

## Nota
He dejado dos bloques para automatizar la descarga todos los días a las 5 de la tarde (1 hora después del cierre del mercado) pero se debe se debe tener en cuenta que si el notebook se cierra, el proceso se detiene por lo cual si desean usarlos deben tener alguna forma de mantener el notebook abierto.